### Getting started with rawML

Here we import some basic methods and functions.

The core rawML library contains jTensors (rML equivalent of torch tensors), createModel method (which is used to stich together different components), and other essentials.

rawML.layers contains all defined layers (relu, linear etc)

rawML.optimzers contains all available optimizers for model parameter tuning

rawML.losses contains all loss functions.

These have been created because rawML losses/optimizers require certain methods to be present to work with the overall flow. A guide on custom layers/optimizers/loss functions shall be released later

In [1]:
import rawML as rML #Core library 
#The first import will return a small greeting :)

#Here we define a jTensor. 
tensor1 = rML.jTensor([1,2,3])


Welcome to rawML


jTensors inherit from numpy.ndarrays, with the primary difference being that they also consist of an attribute (.gd) [strikingly similar to torch.grad], which contains the grads of that particular tensor. 

This is useful during backpropogation and parameters update.

rML supports basic mathematical methods like mean, min, std etc.

In [2]:
print(type(tensor1))
print(tensor1.shape)
print(rML.mean(tensor1))
print(rML.min(tensor1))
print(rML.std(tensor1))


<class 'rawML.backend.jTensor'>
(3,)
2.0
2.0
2.0


To create a rML model, one needs to first define a list of model layers. <br>
Layers can be imported from rawML.layers. We shall take linear and relu for our small example <br>
To create an instance of a Linear layer, one needs to specify the input dimensions and the output dimensions
By default, all linear layers are initialized using He normal initialization. <br>
All layers have the .get_wb() and .show_wb() method, which return and show the weight and bias jTensors corresponding to that layer

In [3]:
from rawML.layers import relu, linear
L1 = linear(3,4)
print(type(L1))
L1.show_wb()

layers_list = [
    L1,
    relu(),
    linear(4,2)
]



<class 'rawML.layers.linear'>
Weight Matrix :-
[[ 0.48748231  1.47880723  0.42344587 -1.49559028]
 [ 0.79058535 -1.3254287  -0.76368347  0.71308381]
 [-0.09159816  1.15006461 -0.32920608 -0.34791512]]
Bias Matrix :-
[[0. 0. 0. 0.]]


The remaining things required to define a rML model are the optimizer, and the loss function. Both can be imported from the respective libraries.

For the Gradient Descent Optimizer, one can get and vary the lr using .get_lr() and .update_lr() methods respectively.

In [5]:
from rawML.losses import MSELoss
from rawML.optimizers import GDOptimizer

loss = MSELoss()
optim = GDOptimizer(5e-3)

Here we combine it all together!
The core rML library comes with a createModel class, which takes input the layers list, optimizer and loss function.<br>
Each layer and loss function have a backward method, which computes the dLoss/d(tensor) corresponding to that particular layer. <br> 
All forward passes, backward passes and parameters updating using optimzer have been wrapped in the model.train method. <br>
Ofcouse, one can set the number of epochs and verbose frequency too!

The model.train mandatorily takes a training split and a validation split. <br>
It has to be passed as (x_train,y_train),(x_val,y_val), as show below.

rML.randn([shape]), and rML.rand([shape]) can be used to create random arrays of the specified dimensions.

Also note that for this demo, we use sklearn's test_train_split to create us a random split.[Yes, they support jTensors] Although the data is random, the purpose of this demo is to give an idea on how rML works, and how to use it as a high level API for model development, training and evaluation

In [6]:
model = rML.createModel(layers_list, optim, loss)

#Creating Random X,Y and split
from sklearn.model_selection import train_test_split as tts
X = rML.randn([20,3])
Y = rML.rand([20,2])
X_train,X_val,Y_train,Y_val = tts(X,Y, train_size=0.8)

In [7]:
model.train((X_train,Y_train), (X_val,Y_val), epochs = 50, verbose_freq= 5)

e:1 | trainLoss: 0.14100, valLoss: 0.38986, optLr: 0.005
e:6 | trainLoss: 0.07721, valLoss: 0.27050, optLr: 0.005
e:11 | trainLoss: 0.05329, valLoss: 0.21080, optLr: 0.005
e:16 | trainLoss: 0.04144, valLoss: 0.17568, optLr: 0.005
e:21 | trainLoss: 0.03430, valLoss: 0.15307, optLr: 0.005
e:26 | trainLoss: 0.02953, valLoss: 0.13762, optLr: 0.005
e:31 | trainLoss: 0.02603, valLoss: 0.12657, optLr: 0.005
e:36 | trainLoss: 0.02288, valLoss: 0.11751, optLr: 0.005
e:41 | trainLoss: 0.02032, valLoss: 0.11068, optLr: 0.005
e:46 | trainLoss: 0.01820, valLoss: 0.10539, optLr: 0.005


This shows how we can access the weights, biases of individual layers. <br>
Note that L1 was defined above, as the first linear layer

One noted issue is that grads cannot be accessed outside train method. I am working on that


In [8]:
L1.show_wb()

Weight Matrix :-
[[ 0.48235223  1.45980461 -0.04983975 -1.38805826]
 [ 0.7908057  -1.31400947 -0.52531809  0.58301378]
 [-0.07586737  1.15576805 -0.17298106 -0.38553533]]
Bias Matrix :-
[[-1.90042869e-04 -1.11061402e-02 -2.64072810e-01 -1.77050586e-01]]


There we go! We successfully trained a model using rawML.
Inference is fairly straightforward, as shown below

In [9]:
y_out = model(rML.randn([50,3]))
print(y_out.shape)

(50, 2)
